In [18]:
from robotic import ry
import numpy as np
import time

print('ry version', ry.__version__, ry.compiled())
ry.params_add({'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_print()

ry version 0.0.19 compile time: Sep 23 2023 14:58:54
-- ry.cpp:operator():99(0) python,
physx/motorKp: 10000,
physx/motorKd: 1000,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/useFCL,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopIters: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
opt/muInit: 1,
opt/muInc: 5,
opt/muMax: 10000,
opt/muLBInit: 0.1,
opt/muLBDec: 0.2,
opt/maxLambda: -1,
opt/constrainedMethod: ,
seed: 0,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/motorKp: 10000,
physx/motorKd: 1000


In [19]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('push_cargo.g')
C.addFile('push_maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

In [20]:
C.view()
way0 = C.addFrame('way0', 'cargo')
way1 = C.addFrame('way1', 'cargo')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(-.07 0 0) d(90 0 0 1)')

way1.setShape(ry.ST.marker, size=[.1])
way1.setRelativePose('t(.2 0 0) d(90 0 0 1)')

In [21]:
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1, 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([1.], ry.FS.poseDiff, ['base', 'way0'], ry.OT.eq, [1e1]);


ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

{ time: 0.00085, evals: 10, done: 1, feasible: 0, sos: 3.47141, f: 0, ineq: 0, eq: 3.20247 }
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
----newton---- initial point f(x):170.579 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    129.293  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    95.0729  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    67.3627  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    45.7313  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    29.8432  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    19.4334  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    14.2815  ACCEPT
--newton-- it:   8  |Delta|:   0.104457  alpha:          1  evals:  

In [23]:
komo.view(False, "waypoints solution")

0

In [24]:
komo.view_close()
path = komo.getPath()

base = C.getFrame('base')

In [30]:
C.setJointState(qHome)
C.view()

0

In [31]:
for t in range(0, path.shape[0]):
    base.setPosition(path[t])
    time.sleep(.1)